## 加载环境变量

In [2]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

## 定义工具

In [3]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool


# 加法工具
class add(BaseModel):
    a: int = Field(..., description="The first number")
    b: int = Field(..., description="The second number")


# 乘法工具
class multiply(BaseModel):
    a: int = Field(..., description="The first number")
    b: int = Field(..., description="The second number")


# 使用装饰器定义工具
@tool
def subtraction(a: int, b: int) -> int:
    """减法工具"""
    return a + b


print(subtraction.name)
print(subtraction.description)
print(subtraction.args)

subtraction
减法工具
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## 大模型的Tool Call工具调用能力

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)

tools = [add, multiply]  # 一个模型可以绑定多个工具
# 将工具绑定到 LLM
llm_with_tools = llm.bind_tools(tools)

query = "3乘以12是多少？"

# 模型将自然语言翻译成调用工具的参数，然后langchain根据参数调用工具
# 得到调用工具的参数
# 调用 LLM 并获取 tool_calls 列表
response = llm_with_tools.invoke(query)

tool_calls = response.tool_calls
# 打印工具调用信息
print(tool_calls)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_70665t5lq35aj9nf967krahl', 'type': 'tool_call'}]
